In [44]:
import configparser
import psycopg2
config = configparser.ConfigParser()
config.read("dwh.cfg")
print(*config["REDSHIFT"].values())

'dwhcluster.cu0kxfsouwy2.us-west-2.redshift.amazonaws.com' dwh dwhuser Passw0rd 5439


In [63]:
connection = psycopg2.connect(
    "host={} dbname={} user={} password={} port={}".format(
        *config["REDSHIFT"].values()
    )
)
cursor = connection.cursor()

In [68]:
query = """
WITH 
    user_log_data AS (
        SELECT
            userId AS user_id,
            firstName AS first_name,
            lastName AS last_name,
            gender,
            level,
            TIMESTAMP 'epoch' + ts/1000 * INTERVAL '1 second' AS time
        FROM
            log_data
        WHERE
            auth = 'Logged In' AND
            length > 0
    ), 
    max_user_times AS (
        SELECT
            user_id,
            MAX(time) AS max_time
        FROM
            user_log_data
        GROUP BY
            user_id
    )
SELECT
    user_log_data.user_id,
    user_log_data.first_name,
    user_log_data.last_name,
    user_log_data.gender,
    user_log_data.level
FROM
    user_log_data
JOIN
    max_user_times
ON
    user_log_data.user_id = max_user_times.user_id AND
    user_log_data.time = max_user_times.max_time;
"""

cursor.execute(query)

rows = cursor.fetchall()
for row in rows:
    print(row)

In [73]:
query = """
WITH 
    user_log_data AS (
        SELECT
            userId AS user_id,
            firstName AS first_name,
            lastName AS last_name,
            gender,
            level,
            TIMESTAMP 'epoch' + ts/1000 * INTERVAL '1 second' AS time
        FROM
            log_data
        WHERE
            auth = 'Logged In' AND
            length > 0
    ), 
    max_user_times AS (
        SELECT
            user_id,
            MAX(time) AS max_time
        FROM
            user_log_data
        GROUP BY
            user_id
    ),
    users_input AS (
        SELECT
            user_log_data.user_id,
            user_log_data.first_name,
            user_log_data.last_name,
            user_log_data.gender,
            user_log_data.level
        FROM
            user_log_data
        JOIN
            max_user_times
        ON
            user_log_data.user_id = max_user_times.user_id AND
            user_log_data.time = max_user_times.max_time
    )
INSERT INTO users FROM users_input;
"""

cursor.execute(query)

rows = cursor.fetchall()
for row in rows:
    print(row)

SyntaxError: syntax error at or near "END"
LINE 41: END WITH
         ^


In [70]:
rows = cursor.execute("SELECT * FROM users;")
for row in rows:
    print(row)

TypeError: 'NoneType' object is not iterable